In [ ]:
import os
import time
import requests
import pandas as pd

In [ ]:
def fetch_zillow_data(api_key, location):
    url = f"https://api.zillow.com/v1/properties"
    headers = {"Authorization": f"Bearer {api_key}"}
    params = {"location": location}
    try:
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching Zillow data: {e}")
        return {}

In [ ]:
def fetch_realtor_data(api_key, location, limit=50):
    url = f"https://realtor.p.rapidapi.com/properties/v2/list-for-sale"
    headers = {
        "X-RapidAPI-Key": api_key,
        "X-RapidAPI-Host": "realtor.p.rapidapi.com",
    }
    params = {"city": location, "limit": limit}
    try:
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching Realtor data: {e}")
        return {}

In [ ]:
def fetch_estated_data(api_key, address):
    url = f"https://apis.estated.com/v4/property"
    params = {
        "token": api_key,
        "address": address,
    }
    response = requests.get(url, params=params)
    return response.json() if response.status_code == 200 else {}

In [ ]:
def fetch_ibge_data(location):
    url = f"https://servicodados.ibge.gov.br/api/v1/cities/{location}"
    response = requests.get(url)
    return response.json() if response.status_code == 200 else {}

In [ ]:
def fetch_google_places_data(api_key, location):
    url = f"https://maps.googleapis.com/maps/api/place/textsearch/json"
    params = {
        "query": location,
        "key": api_key,
    }
    response = requests.get(url, params=params)
    return response.json() if response.status_code == 200 else {}

In [ ]:
def fetch_mapbox_data(api_key, location):
    url = f"https://api.mapbox.com/geocoding/v5/mapbox.places/{location}.json"
    params = {
        "access_token": api_key
    }
    response = requests.get(url, params=params)
    return response.json() if response.status_code == 200 else {}

In [ ]:
def fetch_realtor_data(api_key, location, limit=50):
    url = f"https://realtor.p.rapidapi.com/properties/v2/list-for-sale"
    headers = {
        "X-RapidAPI-Key": api_key,
        "X-RapidAPI-Host": "realtor.p.rapidapi.com",
    }
    params = {"city": location, "limit": limit}
    try:
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching Realtor data: {e}")
        return {}

In [ ]:
def consolidate_data():
    realtor_key = os.getenv("REALTOR_API_KEY")
    zillow_key = os.getenv("ZILLOW_API_KEY")
    google_places_key = os.getenv("GOOGLE_PLACES_API_KEY")
    mapbox_key = os.getenv("MAPBOX_API_KEY")

    locations = ["New York", "Los Angeles", "São Paulo"]
    data_frames = []

    for location in locations:
        print(f"Fetching data for {location}...")
        
        # Fetch data from APIs
        realtor_data = fetch_realtor_data(realtor_key, location)
        zillow_data = fetch_zillow_data(zillow_key, location)
        google_places_data = fetch_google_places_data(google_places_key, location)
        mapbox_data = fetch_mapbox_data(mapbox_key, location)

        # Convert data to DataFrames
        df_realtor = pd.DataFrame(realtor_data.get("properties", []))
        df_zillow = pd.DataFrame(zillow_data.get("listings", []))
        df_google = pd.DataFrame(google_places_data.get("results", []))
        df_mapbox = pd.DataFrame(mapbox_data.get("features", []))

        # Normalize column names for consistency
        for df in [df_realtor, df_zillow, df_google, df_mapbox]:
            df.columns = df.columns.str.lower().str.replace(" ", "_")

        # Append DataFrames
        data_frames.extend([df_realtor, df_zillow, df_google, df_mapbox])

        # Handle rate limits
        time.sleep(1)  # Add delay between API calls to avoid rate limits

    # Concatenate all data into a single DataFrame
    consolidated_df = pd.concat(data_frames, ignore_index=True, sort=False)
    return consolidated_df

In [ ]:
if __name__ == "__main__":
    final_data = consolidate_data()
    final_data.to_csv("real_estate_data.csv", index=False)
    print("Data saved to real_estate_data.csv")